# Project 1: Build an LLM Playground

Welcome to your first project! In this project, you'll build a simple large language model (LLM) playground, an interactive environment where you can experiment with LLMs and understand how they work under the hood.

The goal here is to understand the foundations and mechanics behind LLMs rather than relying on higher-level abstractions or frameworks. You'll see what happens under the hood, how an LLM receives text, processes it, and generates a response. In later projects, you'll use frameworks like `Ollama` and `LangChain` that simplify many of these steps. But before that, this project will help you build a solid mental model of how LLMs actually work.

---
## Environment Setup
We'll use Google Colab, a free browser-based platform that lets you run Python code and machine learning models without installing anything locally. Go to [Google Colab](https://colab.research.google.com/) and upload this notebook to get started.

If you prefer to run the project locally, you need a reproducible setup. Open a terminal in the same directory as this notebook and run the environment setup commands below to install dependencies and create an isolated environment.

### Step 1: Use Conda or uv to install the project dependencies.

#### Option 1: Conda

[Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/index.html) is an open-source package and environment manager that lets you create isolated environments and install Python and non-Python dependencies together.

```bash
# Create and activate the conda environment
conda env create -f environment.yaml && conda activate llm_playground
```

#### Option 2: uv (faster)

[uv](https://docs.astral.sh/uv/) (faster) is a fast Python package installer and virtual environment tool written in Rust that aims to replace pip, pip-tools, and virtualenv with a single, high-performance workflow.

```bash
# Install uv (skip if already installed)
curl -LsSf https://astral.sh/uv/install.sh | sh

# Create venv and install dependencies
uv venv .venv && source .venv/bin/activate
uv pip install -r requirements.txt
```

### Step 2: Register this environment as a Jupyter kernel
This step is optional. Do it only if your environment doesn’t appear in Jupyter’s kernel list.
```bash
python -m ipykernel install --user --name=llm_playground --display-name "llm_playground"
```

Now switch the kernel to `llm_playground` (Kernel → Change Kernel).

---
## Learning Objectives  
- Understand tokenization and how raw text is converted into a sequence of discrete tokens
- Load and inspect a pretrained LLM (GPT-2) using Hugging Face
- Understand logits, probabilities, and how models predict the next token
- Count and explore model parameters to understand model scale
- Explore decoding strategies: greedy decoding and top-p (nucleus) sampling
- See the leap from GPT-2 (simple text completion) to a modern model that understands questions and thinks before answering
- Look ahead: inference engines for serving models in practice


Let's get started!

---

In [ ]:
# Confirm required libraries are installed and working.
import torch, transformers, tiktoken
print("torch", torch.__version__, "| transformers", transformers.__version__)
print("✅ Environment check complete. You're good to go!")

# 1: Tokenization

A neural network cannot process raw text directly. It needs numbers.
Tokenization is the process of converting text into numerical IDs that models can understand. In this section, you will learn how tokenization works in practice and why it is an essential step in every language model pipeline.

Tokenization methods generally fall into three main categories:
1. Word-level
2. Character-level
3. Subword-level

### 1.1 - Word-level tokenization
This method splits text by whitespace and treats each word as a single token. In the next cell, you will implement a basic word-level tokenizer by building a vocabulary that maps words to IDs and writing `encode` and `decode` functions.

In [ ]:
# Creating a tiny corpus. In practice, a corpus is generally the entire internet-scale dataset used for training.

corpus = [
    "Please do not deploy on friday",
    "it works on my machine",
    "extra spicy noodles are a bad idea!",
    "tokens are tiny pieces of text",
]

# Step 1: Build vocabulary (all unique words in the corpus) and mappings
vocab = []
word2id = {}
id2word = {}

# This snippet iterates through all the sentences in the corpus, 
# splits them by whitespace to extract individual words, 
# and then populates your vocab list along with the word2id and id2word dictionaries.
for sentence in corpus:
    # Split each sentence by whitespace to get words
    words = sentence.split()
    for word in words:
        # If we haven't seen this word yet, add it to our vocabulary
        if word not in word2id:
            idx = len(vocab)
            vocab.append(word)
            word2id[word] = idx
            id2word[idx] = word

# Print the results
print("Vocabulary List:")
print(vocab)
print("\nWord to ID mapping:")
print(word2id)
print("\nID to Word mapping:")
print(id2word)


In [ ]:
# Step 2: Define encode and decode functions
def encode(text):
    # converts text to token IDs
    # Split text by whitespace and look up each word's integer ID
    return [word2id[word] for word in text.split()]
    pass


def decode(ids):
    # converts token IDs back to text
    # Look up the word for each ID and join them with a single space
    return " ".join([id2word[idx] for idx in ids])
    pass


In [ ]:
# Step 3: Test your tokenizer with some random sentences.
# 2. Test encoding a string
test_sentence = "tokens are tiny pieces of text"
print("Original Text:", test_sentence)
encoded_ids = encode(test_sentence)
print("Encoded IDs:", encoded_ids)
# 3. Test decoding the IDs back into a string
decoded_text = decode(encoded_ids)
print("Decoded Text:", decoded_text)

While word-level tokenization is simple and easy to understand, it has two key limitations that make it impractical for large-scale models:
1. Large vocabulary size: every new word or variation (for example, run, runs, running) increases the total vocabulary, leading to higher memory and training costs.
2. Out-of-vocabulary (OOV) problem: the model cannot handle unseen or rare words that were not part of the training vocabulary, so they must be replaced with a generic [UNK] token.

The next section introduces character-level tokenization, where text is represented as individual characters instead of words.

### 1.2 - Character-level tokenization

In this approach, every single character (including spaces, punctuation, and even emojis) is assigned its own ID.

In the next cell, we will rebuild a tokenizer using the same corpus as before, but this time with a character-level approach.
For simplicity, we will only use English letters (a-z, A-Z) and punctuation.

In [ ]:
import string

# Step 1: Create a vocabulary that includes uppercase letters, lowercase letters, and punctuation.
vocab = []
char2id = {}
id2char = {}

# We combine letters, punctuation, and a space character
vocab = list(string.ascii_letters + string.punctuation + " ")
for idx, char in enumerate(vocab):
    char2id[char] = idx
    id2char[idx] = char

In [ ]:
# Step 2: Implement encode() and decode() functions to convert between text and IDs.
def encode(text):
    # convert text to list of IDs
    # We only include characters that exist in our vocab.
    return [char2id[char] for char in text if char in char2id]
    pass

def decode(ids):
    # Convert list of IDs to text
    # Join without spaces, as spaces have their own IDs.
    return "".join([id2char[idx] for idx in ids])
    pass


In [ ]:
# Step 3: Test your tokenizer on a some sample words.
test_sentence = "Wow, it works on ALMOST anything!"
encoded_ids = encode(test_sentence)
decoded_text = decode(encoded_ids)
print("Original Text:", test_sentence)
print("Encoded IDs:", encoded_ids)
print("Decoded Text:", decoded_text)

Character-level tokenization solves the out-of-vocabulary problem but introduces new challenges:

1. Longer sequences: because each word becomes many tokens, models need to process much longer inputs.
2. Weaker semantic representation: individual characters carry very little meaning, so models must learn relationships across many steps.
3. Higher computational cost: longer sequences lead to more tokens per input, which increases training and inference time.

To find a better balance between vocabulary size and sequence length, we move to subword-level tokenization next.

### 1.3 - Subword-level tokenization

Subword methods such as `Byte-Pair Encoding (BPE)`, `WordPiece`, and `SentencePiece` **learn** common groups of characters and merge them into tokens. For example, the word **unbelievable** might turn into three tokens: **["un", "believ", "able"]**. This approach strikes a balance between word-level and character-level methods and fixes their limitations.

The BPE algorithm builds a vocabulary iteratively using the following process:
1. Start with individual characters or bytes (each character is a token).
2. Count all adjacent pairs of tokens in a large text corpus.
3. Merge the most frequent pair into new tokens.

Repeat steps 2 and 3 until you reach the desired vocabulary size (for example, 50,000 tokens).

In the next cell, you will experiment with BPE in practice to see how it compresses text into meaningful subword units. Instead of implementing the algorithm from scratch, you will use a pretrained tokenizer, which was already trained on a large text corpus to build its vocabulary, such as the data used to train `GPT-2`. This allows you to see how BPE works in practice with a real, learned vocabulary.

In [ ]:
from transformers import AutoTokenizer

# Step 1: Load a pretrained GPT-2 tokenizer from Hugging Face. 
# Refer to this to learn more: https://huggingface.co/docs/transformers/en/model_doc/gpt2

bpe_tok = AutoTokenizer.from_pretrained("gpt2")


: 

In [ ]:
# Step 2: Encode a sample sentence and inspect how BPE breaks it into subword tokens.
# Hint: bpe_tok has encode, decode, and convert_ids_to_tokens methods.

sample_text = "Tokenization helps models understand text better."
token_ids = bpe_tok.encode(sample_text)
subword_tokens = bpe_tok.convert_ids_to_tokens(token_ids)
decoded_text = bpe_tok.decode(token_ids)

print("Original text:", sample_text)
print("Token IDs:", token_ids)
print("Subword tokens:", subword_tokens)
print("Decoded text:", decoded_text)

### 1.4 - TikToken

`tiktoken` is a fast, production-ready library for tokenization used by OpenAI models.
It is designed for efficiency and consistency with how OpenAI counts tokens in GPT models.

In this section, you will explore how different model families use different tokenizers. We will compare tokenizers used to train `GPT-2` and more powerful models such as `GPT-4`. By trying both, you will see how tokenization has evolved to handle more diverse text (including emojis and special characters) while remaining efficient.

In the next cell, you will use tiktoken to load these encodings and inspect how each one splits the same text. You may find reading this doc helpful: https://github.com/openai/tiktoken

In [ ]:
import tiktoken

# Compare GPT-2 and GPT-4 tokenizers using tiktoken.

# Step 1: Load two tokenizers
gpt2_enc = tiktoken.get_encoding("gpt2")
gpt4_enc = tiktoken.get_encoding("cl100k_base")


# Step 2: Encode the same sentence with both and observe how they differ
sentence = "The 🌟 star-programmer implemented AGI overnight."
gpt2_ids = gpt2_enc.encode(sentence)
gpt4_ids = gpt4_enc.encode(sentence)
print("GPT-2 token IDs:", gpt2_ids)
print("GPT-4 token IDs:", gpt4_ids)
print("GPT-2 token count:", len(gpt2_ids), "| GPT-4 token count:", len(gpt4_ids))


# Step 3: Inspect special tokens used in each tokenizer
print("GPT-2 special tokens:", sorted(gpt2_enc.special_tokens_set))
print("GPT-4 special tokens:", sorted(gpt4_enc.special_tokens_set))
print("GPT-2 vocab size:", gpt2_enc.n_vocab, "| GPT-4 vocab size:", gpt4_enc.n_vocab)

Try changing the input sentence and observe how different tokenizers behave.
Experiment with:
- Emojis, special characters, or punctuation
- Code snippets or structured text
- Non-English text (for example, Japanese, French, or Arabic)

If you are curious, you can also attempt to implement the BPE algorithm yourself using a small text corpus to see how token merges are learned in practice.

### 1.5 - Key Takeaways
- **Word-level**: simple and intuitive, but limited by large vocabularies and out-of-vocabulary issues
- **Character-level**: flexible and covers all text, but produces long sequences that are harder to model
- **Subword / BPE**: balances both worlds and is the default choice for most modern LLMs
- **TikToken**: a production-ready tokenizer used in OpenAI models, demonstrating how optimized subword vocabularies are applied in real systems

# 2: What is a Language Model?

At its core, a **language model** is a function that predicts the next token. Given a sequence of tokens `[t₁, t₂, …, tₙ]`, it outputs probabilities for the next token `tₙ₊₁`. 

Models like GPT-2 use many stacked Transformer layers. Each layer mixes information between tokens (attention) and transforms it (feed-forward). Together, these layers learn patterns in text. At the end, the model produces logits: one score per token in the vocabulary. Higher logits mean the token is more likely to be next.

In this section, you’ll load GPT-2, look at its architecture, count its parameters, and see how it predicts the next token.

### 2.1: Loading GPT-2

There are different ways to load and run pretrained language models.

In this project, we’ll use Hugging Face Transformers, a popular Python library that makes it easy to download models like GPT-2 and run them locally.

There are also dedicated inference engines for serving and running modern LLMs more efficiently, such as **Ollama**, **SGLang**, and **vLLM**. We’ll explore those in future projects.

In the next cell, you’ll load GPT-2 and inspect its architecture.

In [ ]:
import torch
from transformers import GPT2LMHeadModel

# Step 1: Load the smallest GPT-2 model (124M parameters) using the Hugging Face transformers library.
# Refer to: https://huggingface.co/docs/transformers/en/model_doc/gpt2
"""
YOUR CODE HERE (~1 line of code)
"""


# Step 2: Print the full model architecture
"""
YOUR CODE HERE (~1 line of code)
"""


# Step 3: Inspect the first Transformer block by printing its layers.
"""
YOUR CODE HERE (~1 line of code)
"""

### 2.2: Counting Parameters

You often hear that an LLM has a few million or billion parameters. But what does that actually mean? Every weight and bias value inside every layer is a parameter. These are the numbers that the model learned during training.

Next, you will count the total number of parameters in GPT-2 and break them down by component to see where most of the model's capacity lives.

In [ ]:
# Count the total number of parameters in GPT-2
# Hint: use model.parameters() and the .numel() method on each parameter tensor
"""
YOUR CODE HERE (~2-3 lines of code)
"""

**Think about scale:** GPT-2 Small has 124M parameters. GPT-4 is estimated to have over 1 trillion. If each parameter is a 16-bit floating point number (2 bytes), how much memory would you need just to store GPT-2 in RAM? What about a 70B parameter model?

### 2.3: From Text to Predictions

When you pass a sequence of tokens through a language model, it produces a tensor of logits with shape
`(batch_size, seq_len, vocab_size)`.
Each position in the sequence receives a vector of scores representing how likely every possible token is to appear next. By applying a softmax function on the last dimension, these logits can be converted into probabilities that sum to 1.

In the next cell, you will feed a sentence into GPT-2, print the shape of its logits, and display the five most likely next tokens predicted for the final position in the sequence.

In [ ]:
from transformers import AutoTokenizer

# Step 1: Load the GPT-2 tokenizer (the model is already loaded above)
"""
YOUR CODE HERE (~1 line of code)
"""

In [ ]:
# Step 2: Tokenize input text
text = "Hello my name"

"""
YOUR CODE HERE (~1 line of code)
"""

In [ ]:
# Step 3: Pass the input IDs to the model
"""
YOUR CODE HERE (~2-3 lines of code)
"""

In [ ]:
import torch.nn.functional as F

# Step 4: Predict the next token
# Take the logits from the final position, apply softmax to get probabilities,
# and then extract the top 5 most likely next tokens. 
# You may find F.softmax and torch.topk helpful in your implementation.

"""
YOUR CODE HERE (~3-7 lines of code)
"""

### 2.4: Key Takeaway

A language model is not a black box or something mysterious.
It is a large composition of simple, understandable layers such as attention and feed-forward networks, trained together to predict the next token in a sequence.

By learning this next-token prediction task at scale, the model gradually develops an internal understanding of language structure, meaning, and context, which allows it to generate coherent and relevant text.

# 3: Text Generation (Decoding)
Once a language model can predict the next-token probabilities, we can use it to generate text. This is called text generation or decoding.

Conceptually, generation is a loop:

1. Feed the current token sequence into the model.

2. The model outputs a probability distribution over the next token.

3. A decoding algorithm picks the next token from that distribution.

4. Append the chosen token to the sequence and repeat.

In practice, libraries like Hugging Face Transformers provide generate() methods that handle this loop for you, including stopping conditions, batching, and efficiency tricks.

Different decoding strategies control how the next token is chosen, and how creative or deterministic the output feels:

- **Greedy decoding**: always pick the token with the highest probability. Simple and consistent, but often repetitive.

- **Top-p (nucleus) sampling**: randomly sample from the smallest set of tokens whose cumulative probability exceeds a threshold p. This adds variety while keeping outputs coherent.

In the following sections, you'll use generate() to produce text from GPT-2 using both greedy decoding and top-p sampling.

### 3.1: Greedy Decoding
In this section, you will use GPT-2 and Hugging Face's built-in generate method to produce text using the greedy decoding strategy.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "gpt2"

# Step 1. Load GPT-2 model and tokenizer.
"""
YOUR CODE HERE (~2 lines of code)
"""


# Step 2. tokenize the prompt, then use model.generate() to generate text, then print the text
prompt = "Hi my name"

"""
YOUR CODE HERE (~3-6 lines of code)
"""

In [ ]:
# Step 3: Write a helper function that wraps model.generate for greedy decoding,
# then test it on multiple prompts to observe repetition patterns.

tests = ["Once upon a time", "What is 2+2?", "Suggest a party theme."]

def generate(model, tokenizer, prompt, max_new_tokens=32):
    """
    YOUR CODE HERE (~5 lines of code)
    """
    pass

for prompt in tests:
    print(f"\n GPT-2 | Greedy")
    print(generate(model, tokenizer, prompt, 80))

Naively selecting the single most probable token at each step (known as greedy decoding) often leads to poor results in practice:
- Repetition loops: phrases like "The cat is is is…"
- Short-sighted choices: the most likely token right now might lead to incoherent text later

These issues are why more advanced decoding methods such as top-p (nucleus) sampling are commonly used to make model outputs more diverse and natural.

### 3.2: Top-p (Nucleus) Sampling
Top-p sampling (also called nucleus sampling) introduces controlled randomness into text generation. Instead of always picking the single most likely token, it samples from the smallest set of tokens whose cumulative probability exceeds a threshold `p` (e.g., 0.9).

This allows the model to explore multiple plausible continuations, producing more diverse and natural-sounding text while still staying coherent.

In this section, you will implement a generate function that supports both greedy and top-p strategies.

In [ ]:
# Use model.generate to use top-p instead of greedy
# You may find this link helpful: https://huggingface.co/docs/transformers/en/main_classes/text_generation
prompt = "Hi my name"

"""
YOUR CODE HERE (~5 lines of code)
"""


In [ ]:
# Write a helper function that wraps model.generate for top-p decoding,
# then test it on multiple prompts
def generate(model, tokenizer, prompt, strategy="top_p", max_new_tokens=128):
    """
    YOUR CODE HERE (~10-15 lines of code)
    """
    pass


tests=["Once upon a time","What is 2+2?", "Suggest a party theme."]
for prompt in tests:
    print(f"\n== GPT-2 | Top-p ==")
    print(generate(model, tokenizer, prompt, "top_p", 32))

# 4: From Simple GPT2 to Modern LLMs

So far, we have used **GPT-2**, one of the earliest publicly available language models (released in 2019, 124M parameters). GPT-2 can only do one thing: **complete text**. Given some input, it predicts what words come next. It has no concept of questions, instructions, or conversation. If you type `"What is 2+2?"`, GPT-2 will continue the text as if it were part of a web page or article. It does not understand you are asking a question.

Modern language models are a different. Models like **Qwen3-0.6B** (2025, 600M parameters) have gone through additional training stages that unlock fundamentally new capabilities:

- **Instruction following**: they interpret your input as a request and produce a helpful response
- **Conversation**: they maintain context across a multi-turn dialogue
- **Reasoning**: they can *think step-by-step* before answering, using internal reasoning tokens (`<think>...</think>`) to work through a problem before giving a final answer

In this section, you will see this dramatic contrast firsthand by giving the same prompts to both models. In week 4, we'll learn about thinking and reasoning models in detail.

### 4.1: Chat Templates

Instruction-tuned models expect input in a structured **chat format**, not raw text. Instead of receiving `"What is 2+2?"` as plain text, the model expects a formatted message like:

```
<|user|>What is 2+2?<|assistant|>
```

Each model family defines its own format. The Hugging Face `tokenizer.apply_chat_template()` method handles this formatting automatically. Without it, even an instruction-tuned model receives unstructured text and falls back to simple completion behavior.

### 4.2: GPT-2 vs. Qwen3-0.6B

In the next cells, you will load both models and feed the same prompt to each one:

- **GPT-2**: receives the raw prompt and blindly continues the text
- **Qwen3-0.6B**: receives a properly formatted chat message, *thinks* through the problem, and produces a direct answer

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load both GPT-2 and Qwen/Qwen3-0.6B models using HuggingFace `.from_pretrained` method.
"""
YOUR CODE HERE (~5-15 lines of code)
"""

Both models are now loaded. GPT-2 has 124M parameters; Qwen3-0.6B has roughly 600M. If the previous cell took some time, that was mainly due to model download. The models are cached locally, so future runs will be much faster.

Next, we will generate text from both models using the same prompt. For GPT-2, we pass the raw text directly. For Qwen, we first format the prompt as a chat message using `apply_chat_template()`, then generate. Both use **top-p sampling** so the outputs are varied and natural.

In [ ]:

# Generate text from both models using the same prompt.
# For GPT-2, pass the raw prompt directly.
# For Qwen, use apply_chat_template() to format it as a chat message before generating.
# Refer to: https://huggingface.co/docs/transformers/en/chat_templating
# Use top-p (nucleus) sampling for both models.

prompt = "What is 2+2?"

# --- GPT-2: simple text completion ---

"""
YOUR CODE HERE (~5 lines of code)
"""

# --- Qwen3-0.6B: instruction-tuned with chat template ---

"""
YOUR CODE HERE (~5 lines of code)
"""

# 5. (Optional) A Small Interactive LLM Playground
This section is optional. You do not need to implement it to complete the project. It is meant purely for exploration and will not significantly affect your core AI engineering skills.

If you are curious, you can build a simple interactive playground to experiment with text generation. You can:
- Create input widgets for the prompt, model selection, decoding strategy, and temperature
- Use Hugging Face's generate method to produce text based on the selected settings
- Display the model's response directly in the notebook output

You may find following links helpful:
- https://ipywidgets.readthedocs.io/en/latest/
- https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown


# Steps to implement:
# 1. Load models and tokenizers (GPT-2 and Qwen).
# 2. Define a helper function to generate text with different decoding strategies.
# 3. Create interactive UI elements (prompt box, model selector, strategy selector, temperature slider).
# 4. Add a button to trigger text generation.
# 5. Define the button’s behavior.
# 6. Display the full UI for the playground.

"""
YOUR CODE HERE
"""


# 6: Inference Engines: Ollama, vLLM, SGLang

So far, we loaded models directly in Python using HuggingFace's `transformers` library. This is great for learning, but in practice models run as **servers** that expose an API. Client applications send requests and receive responses over HTTP — the model itself stays loaded in memory (and on the GPU) between requests.

An **inference engine** handles all the heavy lifting: model loading, GPU memory management, request batching, and serving an HTTP API. Popular inference engines include:

| Engine | Best for |
|--------|----------|
| **Ollama** | Easy local use and experimentation |
| **vLLM** | High-throughput production serving |
| **SGLang** | Fast serving + structured output |

Most inference engines expose an **OpenAI-compatible API**. This means you can learn one client library (the `openai` Python package) and swap backends freely: Ollama for local development, vLLM or SGLang for production.

In future weeks, we'll learn about Ollama, set it up, and use it to easily load and build on top of modern powerful LLMs!

## 🎉 Congratulations!

You've just explored the internals of a real **LLM**. In this project you:
* Learned how **tokenization** works — from word-level to BPE — and why it matters
* Used `tiktoken` to compare tokenizers across different model generations
* Loaded GPT-2 and inspected its Transformer blocks and layers
* **Counted parameters** and understood where a model's capacity lives
* Learned how the model produces **logits and probabilities** to predict the next token
* Explored **decoding strategies**: greedy decoding and top-p (nucleus) sampling
* Witnessed the leap from GPT-2 (simple text completion) to Qwen3-0.6B — a modern model that **understands questions and thinks before answering**
* Learned about **inference engines** (Ollama, vLLM, SGLang) and the OpenAI-compatible API pattern

👏 **Great job!** Take a moment to celebrate. You now have a working mental model of how LLMs work — from raw text input all the way to generated output. The skills and intuitions you built here will serve as the foundation for everything that comes next.

